In [1]:
import time
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

from harmonic import HarmonicUpdater

#### initializing snapshot

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=20)
sim.create_state_from_gsd(filename='./DATA/lattice.gsd', frame=-1)

#### initializing integrator

In [3]:
lattice = gsd.hoomd.open('./DATA/lattice.gsd', 'rb')
verts = lattice[-1].particles.type_shapes[0]['vertices']

mc = hoomd.hpmc.integrate.ConvexPolyhedron()
mc.shape["A"] = dict(vertices=verts)

#### harmonic constraint

In [4]:
lattice = gsd.hoomd.open('./DATA/lattice.gsd', 'rb')
pos = lattice[-1].particles.position
ori = lattice[-1].particles.orientation
box = lattice[-1].configuration.box

k_init=90
k_second=0

k_rot_init=10
k_rot_second=0

t_ramp=int(1e5)
k_trans = hoomd.variant.Power(A=k_init, 
                              B=k_second,
                              power=1,
                              t_start=sim.timestep, 
                              t_ramp=t_ramp)
k_rot = hoomd.variant.Power(A=k_rot_init, 
                            B=k_rot_second,
                            power=1,
                            t_start=sim.timestep, 
                            t_ramp=t_ramp)

harmonic = hoomd.hpmc.external.field.Harmonic(reference_positions=pos,
                                              reference_orientations=ori,
                                              k_translational=k_trans,
                                              k_rotational=k_rot,
                                              symmetries=ori)
mc.external_potential=harmonic

#### harmonic updater

In [5]:
harmonicupdater = hoomd.update.CustomUpdater(action=HarmonicUpdater(ref_pos=pos, 
                                                                    init_size=box[0]), trigger=1)

#### constant pressure updater

In [6]:
boxmc = hoomd.hpmc.update.BoxMC(trigger=hoomd.trigger.Periodic(10),
                                   betaP=20)

boxmc.volume['weight']=1.0
boxmc.volume['mode']='standard'
boxmc.volume['delta']=0.1

tune_boxmc = hoomd.hpmc.tune.BoxMCMoveSize.scale_solver(boxmc=boxmc,
                                                        trigger=hoomd.trigger.Periodic(100),
                                                        moves=['volume'],
                                                        target=0.2,
                                                        max_move_size={'volume':1})

#### alchemical updater

In [7]:
vertex_move = hoomd.hpmc.shape_move.Vertex(default_step_size= 0.002, 
                                           vertex_move_probability = 0.2)
vertex_move.volume["A"] = 1

updater = hoomd.hpmc.update.Shape(shape_move=vertex_move, 
                                  trigger=hoomd.trigger.Periodic(1))
updater.shape_move = vertex_move

#### logger for writing gsd

In [8]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

gsd_writer = hoomd.write.GSD(filename='./DATA/trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             mode='wb',
                             filter=hoomd.filter.All(),
                             log=logger)

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/write/gsd.py:183: FutureWarning: log keyword arguments passed to <hoomd.write.gsd.GSD object at 0x12c3305b0> is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


#### attaching operations

In [9]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = mc
sim.operations.updaters.append(updater)
sim.operations.updaters.append(harmonicupdater)
sim.operations.updaters.append(boxmc)
sim.operations.tuners.append(tune_boxmc)

#### run simulation

In [10]:
start = time.time()

sim.run(2e5)

print('Time elapsed', time.time()-start, 's')

Time elapsed 151.58479595184326 s


Failed to solve horizon edge.


#### checking overlaps

In [11]:
mc.overlaps

0